In [1]:
#r "nuget:System.Device.Gpio,1.1.0-prerelease.20276.1"
#r "nuget:IoT.Device.Bindings,1.1.0-prerelease.20276.1 "
#r "nuget:UnitsNet,4.68.0"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package UnitsNet version 4.68.0

Installed package System.Device.Gpio version 1.1.0-prerelease.20276.1

Installed package IoT.Device.Bindings version 1.1.0-prerelease.20276.1

In [3]:
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}


(6,9): warning CS1702: Assuming assembly reference 'System.Drawing.Primitives, Version=4.2.0.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' used by 'Iot.Device.Bindings' matches identity 'System.Drawing.Primitives, Version=4.2.1.0, Culture=neutral, PublicKeyToken=b03f5f7f11d50a3a' of 'System.Drawing.Primitives', you may need to supply runtime policy



In [3]:
async IAsyncEnumerable<double> ReadTemperatureAsync(int count = 10){   
    using (var sh = new SenseHat())
    {
        for (var i = 0; i < count; i++){
            yield return (sh.Temperature2.DegreesFahrenheit);
            await Task.Delay(60000);
        }
    };
}

In [4]:
using System;
var temperatures = new List<(DateTime dt, double val)>();


In [5]:
DateTime RoundUp(DateTime dt, TimeSpan d)
{
    return new DateTime((dt.Ticks + d.Ticks - 1) / d.Ticks * d.Ticks, dt.Kind);
}

In [ ]:
await foreach (var result in ReadTemperatureAsync(15))
{
    var t = RoundUp(DateTime.Now, TimeSpan.FromMinutes(1));
    Console.WriteLine(t + " : " + result);
    temperatures.Add((t, result));
}

9/8/2020 5:22:00 AM : 102.75875091552734
9/8/2020 5:23:00 AM : 103.03250274658203
9/8/2020 5:24:00 AM : 103.23500137329101
9/8/2020 5:25:00 AM : 103.04000091552734
9/8/2020 5:26:00 AM : 102.98375091552734
9/8/2020 5:27:00 AM : 103.39249725341797
9/8/2020 5:28:00 AM : 103.59875183105468
9/8/2020 5:29:00 AM : 103.71500091552734
9/8/2020 5:30:00 AM : 103.57625045776368
9/8/2020 5:31:00 AM : 103.65125274658203


In [ ]:
#r "nuget:Microsoft.Azure.CognitiveServices.AnomalyDetector"

In [ ]:
using Microsoft.Azure.CognitiveServices.AnomalyDetector;
using Microsoft.Azure.CognitiveServices.AnomalyDetector.Models;
using System.Collections;
using Point = Microsoft.Azure.CognitiveServices.AnomalyDetector.Models.Point;

Request GetSeries(List<(DateTime dt, double val)> list)
{
    var points = list
        // .TakeLast(take)
        .Select(e => new Point(e.dt,e.val)).ToList();
    Console.WriteLine(points);
    return new Request(points, Granularity.Minutely, period: 1); 
}

In [ ]:
async Task<bool> LastDetectSampleAsync(IAnomalyDetectorClient client, Request request)
{

    Console.WriteLine("Detecting the anomaly status of the latest point in the series.");
    LastDetectResponse result = await client.LastDetectAsync(request);
    return result.IsAnomaly;
}

To get Endpoint and Key values, create Anomaly Detector in Azure Portal. https://ms.portal.azure.com/#create/Microsoft.CognitiveServicesAnomalyDetector


In [ ]:
using System;
using System.IO;
using System.Text;

string path = "config.json";
string readText = File.ReadAllText(path);
var dict = System.Text.Json.JsonSerializer.Deserialize<Dictionary<string, string>>(readText);

string endpoint = dict["endpoint"];
string key = dict["subscriptionkey"];


In [ ]:
IAnomalyDetectorClient client = new AnomalyDetectorClient(new ApiKeyServiceClientCredentials(key))
{
    Endpoint = endpoint
};

In [ ]:
foreach (var temp in temperatures)
{
    Console.WriteLine(temp.dt + " : " + temp.val);
}
var lastTemp = temperatures.Last();
temperatures.Add((lastTemp.dt.AddMinutes(1) , 200));

var request = GetSeries(temperatures); // The request payload with points from the data file
var isAnomaly = await LastDetectSampleAsync(client, request);
Console.WriteLine($"Anomaly {isAnomaly}");
if (isAnomaly)
{
    using (var ledMatrix = new SenseHatLedMatrixI2c())
    {
            ledMatrix.Fill(Color.Red);
    }
}


In [ ]:
using Iot.Device.SenseHat;
using System;
using System.Drawing;
using (var ledMatrix = new SenseHatLedMatrixI2c())
{
        ledMatrix.Fill(Color.Black);
}